In [1]:
import numpy as np
import pandas as pd 
import pandas_profiling as pp
import plotly.express as px 
import sqlite3
import random
from sklearn.metrics import mean_squared_error

pd.options.display.max_columns = None

## Import des datasets 

In [2]:
df_aeroports = pd.read_parquet("../data/aggregated_data/aeroports.gzip")
df_compagnies = pd.read_parquet("../data/aggregated_data/compagnies.gzip")
df_vols = pd.read_parquet("../data/aggregated_data/vols.gzip")
df_fuel = pd.read_parquet("../data/aggregated_data/prix_fuel.gzip")
df_test = pd.read_parquet("../data/extracted/test_data/vols.gzip")

In [3]:
df_vols.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4332914 entries, 0 to 4332913
Data columns (total 31 columns):
 #   Column                                         Dtype         
---  ------                                         -----         
 0   IDENTIFIANT                                    int64         
 1   VOL                                            int64         
 2   CODE AVION                                     object        
 3   AEROPORT DEPART                                object        
 4   AEROPORT ARRIVEE                               object        
 5   DEPART PROGRAMME                               int64         
 6   HEURE DE DEPART                                float64       
 7   RETART DE DEPART                               float64       
 8   TEMPS DE DEPLACEMENT A TERRE AU DECOLLAGE      float64       
 9   DECOLLAGE                                      float64       
 10  TEMPS PROGRAMME                                float64       
 11  TEMPS PASSE

In [4]:
df_vols['COMPAGNIE AERIENNE']

0            MAF
1            I6F
2          NVPPA
3          NVPPA
4            THA
           ...  
4332909    SMITH
4332910    SMITH
4332911    SMITH
4332912    SMITH
4332913    SMITH
Name: COMPAGNIE AERIENNE, Length: 4332914, dtype: object

In [5]:
df_compagnies

,COMPAGNIE,CODE,NOMBRE D EMPLOYES,CHIFFRE D AFFAIRE
0,Try Hard Airlines,THA,12909,2310000000
1,Corporate Overlord Airways,COA,130595,40579000000
2,Neverland Airlines,NA,18567,7651000000
3,Always A Problem Flights,AAPF,92000,51000000000
4,Overpriced Air,OA,12200,6649000000
5,Morally Ambiguious Fligthts,MAF,15150,6391000000
6,Fliying Is Possible Inc.,FIPI,13230,2979000000
7,Air Piloter Sans Les Mains,APSLM,13112,7651000000
8,Ne Va Pas Partout Airlines,NVPPA,12000,6235000000
9,IE 6.0 Flights,I6F,17569,2660000000


In [6]:
df_vols['COMPAGNIE AERIENNE']

0            MAF
1            I6F
2          NVPPA
3          NVPPA
4            THA
           ...  
4332909    SMITH
4332910    SMITH
4332911    SMITH
4332912    SMITH
4332913    SMITH
Name: COMPAGNIE AERIENNE, Length: 4332914, dtype: object

In [7]:
list_compagnies = df_compagnies['COMPAGNIE'].unique().tolist()
list_airports = df_aeroports['CODE IATA'].unique().tolist()

In [8]:
airlines = [random.choice(list_compagnies) for i in range(100)]
airports = [random.choice(list_airports) for i in range(100)]
nb_passagers = list(random.sample(range(0, 2505), 100))

In [9]:
y_true = np.array(random.sample(range(-100, 1898), 100))
y_preds = np.array(random.sample(range(-100, 1898), 100))
prediction = pd.DataFrame({'RETARD': y_preds, 'COMPAGNIES': airlines, 'AEROPORTS': airports,
                          'NOMBRE DE PASSAGERS': nb_passagers, 'RETARD REEL' : y_true})

# METRICS 

On a choisit la **RMSE** : La racine de l'erreur quadratique moyenne ou racine de l'écart quadratique moyen est une mesure fréquemment utilisée des différences entre les valeurs prédites par un modèle ou estimateur et les valeurs observées

- **Formule** : $RMSE = \sqrt{MSE(\theta)} = \sqrt{\dfrac{1}{n}\sum^{n}_{i=1}\left(y-y_{pred}\right)^{2}}$

- **Interpretation** : plus la valeur de notre RMSE est grande plus notre prédiction (le retard prédit à l'arrivée) est loin de la réalité terrain, du retard à l'arrivée effectif. Une RMSE proche de 0 signifie que notre modèle est proche de la réalité dans ces prédictions. 

On a également choisi une deuxième métrique : 
**Moyenne** : $\dfrac{1}{n}\sum^{n}_{i=1}\left(y-y_{pred}\right)$

**Objectif** : pénaliser l'écart entre la prédiction et la réalité terrain mais aussi pénaliser la direction de l'erreur, c'est à dire si notre modèle prédit plus de retard que prévu ou à l'inverse moins de retard que prévu. 

In [10]:
airline_list = list(dict.fromkeys(airlines))
rmse = []
mean_error=[]
for idx, airline in enumerate(airline_list): 
    y_true = prediction[prediction['COMPAGNIES'] == airline]['RETARD REEL']
    y_preds =  prediction[prediction['COMPAGNIES'] == airline]['RETARD']
    rmse.append(mean_squared_error(y_true, y_preds, squared=False)) #if squared=True return MSE value
    mean_error.append((y_true - y_preds).mean())

df_metrics = pd.DataFrame({'COMPAGNIE':airline_list, 'RMSE': rmse, 'MEAN ERROR': mean_error})

In [11]:
df_metrics

,COMPAGNIE,RMSE,MEAN ERROR
0,Air Penguin,778.446048,469.250000
1,IE 6.0 Flights,899.128149,249.142857
2,Always A Problem Flights,707.021511,213.750000
3,Try Hard Airlines,835.327573,-311.230769
4,Overpriced Air,1014.587207,-177.200000
5,Ne Va Pas Partout Airlines,801.652214,450.818182
6,Air Piloter Sans Les Mains,898.748018,317.500000
7,Better Take A Train Airlines,718.257475,-353.000000
8,Neverland Airlines,666.341204,-108.200000
9,Corporate Overlord Airways,788.401547,-65.400000


# KPIs

In [12]:
prediction["CHIFFRE D'AFFAIRE COMPAGNIE"] = prediction['COMPAGNIES'].map(lambda x:\
                                       df_compagnies[df_compagnies['COMPAGNIE'] ==x]['CHIFFRE D AFFAIRE'].values[0])

In [13]:
prediction_avec_retard = prediction[prediction['RETARD']>0].copy()

### Prix du retard

Hypothese prix retard aéroport (centaine d'euros) : 
- après 10min :  la compagnie paye toutes les minutes le prix indiqué dans la colonne "PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES"
- après 20min : la compagnie paye un **supplément** qui est le prix indiqué dans la colonne "PRIX RETARD PREMIERE 20 MINUTES"

In [14]:
print(len(prediction_avec_retard))

95


In [15]:
prediction_avec_retard

,RETARD,COMPAGNIES,AEROPORTS,NOMBRE DE PASSAGERS,RETARD REEL,CHIFFRE D'AFFAIRE COMPAGNIE
0,1469,Air Penguin,STR,2279,1501,31064000000
1,288,IE 6.0 Flights,TBZ,829,989,2660000000
2,887,Always A Problem Flights,PKX,2355,847,51000000000
3,1282,Try Hard Airlines,CAG,2037,1045,2310000000
4,1529,Overpriced Air,SAW,76,-72,6649000000
...,...,...,...,...,...,...
95,741,Try Hard Airlines,NCE,1095,1554,2310000000
96,341,IE 6.0 Flights,DKR,988,1707,2660000000
97,1736,Bel Air,DMK,1296,560,3671000000
98,1347,Ne Va Pas Partout Airlines,LJU,2315,1242,6235000000


In [16]:
test = pd.merge(prediction_avec_retard, 
                df_aeroports[['CODE IATA', 'PRIX RETARD PREMIERE 20 MINUTES']].rename(columns={'CODE IATA': 'AEROPORTS'}),
                on='AEROPORTS', how='left')
test 

,RETARD,COMPAGNIES,AEROPORTS,NOMBRE DE PASSAGERS,RETARD REEL,CHIFFRE D'AFFAIRE COMPAGNIE,PRIX RETARD PREMIERE 20 MINUTES
0,1469,Air Penguin,STR,2279,1501,31064000000,33
1,288,IE 6.0 Flights,TBZ,829,989,2660000000,89
2,887,Always A Problem Flights,PKX,2355,847,51000000000,24
3,1282,Try Hard Airlines,CAG,2037,1045,2310000000,88
4,1529,Overpriced Air,SAW,76,-72,6649000000,58
...,...,...,...,...,...,...,...
90,741,Try Hard Airlines,NCE,1095,1554,2310000000,53
91,341,IE 6.0 Flights,DKR,988,1707,2660000000,35
92,1736,Bel Air,DMK,1296,560,3671000000,54
93,1347,Ne Va Pas Partout Airlines,LJU,2315,1242,6235000000,70


In [17]:
print(len(test))

95


In [18]:
def add_cost_20min_delay(df_aeroports, airport):
    twenty_first_min_cost = df_aeroports[
        df_aeroports['CODE IATA'] == airport]['PRIX RETARD PREMIERE 20 MINUTES'].values[0]
    return twenty_first_min_cost

def add_cost_10min_delay(df_aeroports, airport):
    ten_min_delay_cost = df_aeroports[
            df_aeroports['CODE IATA'] == airport]['PRIS RETARD POUR CHAQUE MINUTE APRES 10 MINUTES'].values[0]
    return ten_min_delay_cost

In [19]:
def cost_of_delay(pred_vol):
    delay = pred_vol['RETARD']
    twenty_first_min_cost = pred_vol['PRIX RETARD PREMIERE 20 MINUTES']
    ten_min_delay_cost = pred_vol['PRIS RETARD CHAQUE MINUTE APRES 10 MINUTES']
    
    cost = 0
    if delay > 10 : 
        cost += ten_min_delay_cost * (delay - 10) 
    if delay >= 20 : 
        cost += twenty_first_min_cost
    return cost

In [20]:
prediction_avec_retard['PRIX RETARD PREMIERE 20 MINUTES'] = prediction_avec_retard['AEROPORTS']\
                                                            .map(lambda x: add_cost_20min_delay(df_aeroports, x))
    
prediction_avec_retard['PRIS RETARD CHAQUE MINUTE APRES 10 MINUTES'] = prediction_avec_retard['AEROPORTS']\
                                                                .map(lambda x: add_cost_10min_delay(df_aeroports, x))
prediction_avec_retard['COUT DU RETARD'] = prediction_avec_retard.apply(cost_of_delay, axis=1)

In [21]:
prediction_avec_retard = prediction_avec_retard.drop(
    columns=['PRIX RETARD PREMIERE 20 MINUTES', 'PRIS RETARD CHAQUE MINUTE APRES 10 MINUTES'])

### Indemnisation des clients 

Hypothèse : 
- 10% des clients vont demander à être indemnisé pour un retard compris entre 10min et 45min
    - Indemnité à payer : 1/4 du prix du billet
- 20% des clients vont demander à être indemnisé pour un retard supérieur à 1h 
    - Indemnité à payer : 1/2 du prix du billet
- 50% des clients vont demander à être indemnisé pour un retard supérieur à 3h 
    - Indemnité à payer : totalité du prix du billet
    
On fait l'hypothèse d'un fixe maximal du prix du billet : **300€**

In [22]:
def get_number_of_indemnities_asked(pred_vol): 
    delay = pred_vol.loc['RETARD']
    nb_of_passenger = pred_vol.loc['NOMBRE DE PASSAGERS']
    nb_of_indemnities_asked = 0
    if delay > 10 and delay <45: 
        nb_of_indemnities_asked = 20*nb_of_passenger//100
    elif delay > 60 and delay <180:
        nb_of_indemnities_asked = 50*nb_of_passenger//100
    elif delay > 180:
        nb_of_indemnities_asked = 75*nb_of_passenger//100
    return nb_of_indemnities_asked

def compensation_due(pred_vol, ticket_price=300): 
    delay = pred_vol.loc['RETARD']
    nb_of_indemnities_asked = pred_vol.loc["NOMBRE D'INDEMNITES DEMANDEES"]
    compensation_due_to_clients = 0
    if delay > 10 and delay <45: 
        compensation_due_to_clients = (ticket_price/3)*nb_of_indemnities_asked
    elif delay > 60 and delay <180:
        compensation_due_to_clients = (ticket_price/2)*nb_of_indemnities_asked
    elif delay > 180:
        compensation_due_to_clients = ticket_price*nb_of_indemnities_asked
    return compensation_due_to_clients

In [23]:
prediction_avec_retard[
    "NOMBRE D'INDEMNITES DEMANDEES"] = prediction_avec_retard.apply(get_number_of_indemnities_asked, axis=1)
prediction_avec_retard[
    "INDEMNITES A PAYER"] = prediction_avec_retard.apply(compensation_due, axis=1)

## Perte de client : 

**Hypothèse** : Taux d'attrition à 3% pour un retard de plus de 3h 

In [24]:
def get_number_of_lost_customer(delay, passenger_nb):
    if delay > 180 : 
        return passenger_nb*3//100
    else : return 0

prediction_avec_retard['NOMBRE DE CLIENTS PERDUS'] = prediction_avec_retard.apply(
    lambda x: get_number_of_lost_customer(x["RETARD"], x['NOMBRE DE PASSAGERS']), axis=1)

In [25]:
prediction_avec_retard

,RETARD,COMPAGNIES,AEROPORTS,NOMBRE DE PASSAGERS,RETARD REEL,CHIFFRE D'AFFAIRE COMPAGNIE,COUT DU RETARD,NOMBRE D'INDEMNITES DEMANDEES,INDEMNITES A PAYER,NOMBRE DE CLIENTS PERDUS
0,1469,Air Penguin,STR,2279,1501,31064000000,13164,1709,512700.0,68
1,288,IE 6.0 Flights,TBZ,829,989,2660000000,1757,621,186300.0,24
2,887,Always A Problem Flights,PKX,2355,847,51000000000,3532,1766,529800.0,70
3,1282,Try Hard Airlines,CAG,2037,1045,2310000000,10264,1527,458100.0,61
4,1529,Overpriced Air,SAW,76,-72,6649000000,1577,57,17100.0,2
...,...,...,...,...,...,...,...,...,...,...
95,741,Try Hard Airlines,NCE,1095,1554,2310000000,6632,821,246300.0,32
96,341,IE 6.0 Flights,DKR,988,1707,2660000000,2683,741,222300.0,29
97,1736,Bel Air,DMK,1296,560,3671000000,6958,972,291600.0,38
98,1347,Ne Va Pas Partout Airlines,LJU,2315,1242,6235000000,4081,1736,520800.0,69


### Get the cost of all the lost client for the airlines

**Hypothèse** : 

On suppose qu'un client prend en moyenne 3 fois l'avion par an avec la même compagnie (on suppose une fidéité total des clients auprès de leur compagnie).

Donc si la compagnie perd un client, elle perd un cout de **3x"prix du billet"** par client

On suppose le prix du billet = 300€

In [26]:
def get_cost_of_lost_customer(nb_of_lost_customers, ticket_price=300, flight_frequency=3):
    return flight_frequency*ticket_price*nb_of_lost_customers

In [27]:
prediction_avec_retard['COUT DES CLIENTS PERDUS'] = prediction_avec_retard["NOMBRE DE CLIENTS PERDUS"].map(
                                                                        lambda x: get_cost_of_lost_customer(x))

In [28]:
prediction_avec_retard

,RETARD,COMPAGNIES,AEROPORTS,NOMBRE DE PASSAGERS,RETARD REEL,CHIFFRE D'AFFAIRE COMPAGNIE,COUT DU RETARD,NOMBRE D'INDEMNITES DEMANDEES,INDEMNITES A PAYER,NOMBRE DE CLIENTS PERDUS,COUT DES CLIENTS PERDUS
0,1469,Air Penguin,STR,2279,1501,31064000000,13164,1709,512700.0,68,61200
1,288,IE 6.0 Flights,TBZ,829,989,2660000000,1757,621,186300.0,24,21600
2,887,Always A Problem Flights,PKX,2355,847,51000000000,3532,1766,529800.0,70,63000
3,1282,Try Hard Airlines,CAG,2037,1045,2310000000,10264,1527,458100.0,61,54900
4,1529,Overpriced Air,SAW,76,-72,6649000000,1577,57,17100.0,2,1800
...,...,...,...,...,...,...,...,...,...,...,...
95,741,Try Hard Airlines,NCE,1095,1554,2310000000,6632,821,246300.0,32,28800
96,341,IE 6.0 Flights,DKR,988,1707,2660000000,2683,741,222300.0,29,26100
97,1736,Bel Air,DMK,1296,560,3671000000,6958,972,291600.0,38,34200
98,1347,Ne Va Pas Partout Airlines,LJU,2315,1242,6235000000,4081,1736,520800.0,69,62100


# TODO : 

- [x] Rajouter colonne avec pourcentage du chiffre d'affaire perdu 
- [x] Afficher nombre de vols en retard par compagnie 
- [x] combien de clients ils vont perdre : plus de 3h de retard --> taux d'attrition de 5% des clients
- [x] fréquence moyenne de réservation de vol avec la compagnie (3fs/an) : 5%x3x(prix du billet)
- [x] afficher le détail répartition des couts dans total à payer dans les graphes
- [ ] NEXT STEPS 

# BILAN : TOTAL A PAYER

In [29]:
cost_of_delay_gb_airlines = prediction_avec_retard[["RETARD", "COMPAGNIES","CHIFFRE D'AFFAIRE COMPAGNIE",
                                                    "COUT DU RETARD", "INDEMNITES A PAYER", "NOMBRE DE CLIENTS PERDUS", 
                                                    "COUT DES CLIENTS PERDUS"]]\
                            .groupby(['COMPAGNIES'], as_index=False)\
                            .agg({
                                "RETARD" : "count",
                                "CHIFFRE D'AFFAIRE COMPAGNIE":'first',
                                "COUT DU RETARD":'sum',
                                "INDEMNITES A PAYER":'sum',
                                "NOMBRE DE CLIENTS PERDUS": "sum", 
                                "COUT DES CLIENTS PERDUS": "sum"
                            }).rename(columns={"RETARD" : "NOMBRE DE RETARD"})

In [30]:
cost_of_delay_gb_airlines["TOTAL A PAYER"] = cost_of_delay_gb_airlines["COUT DU RETARD"]\
                                            + cost_of_delay_gb_airlines["INDEMNITES A PAYER"]\
                                            + cost_of_delay_gb_airlines["COUT DES CLIENTS PERDUS"]

In [31]:
cost_of_delay_gb_airlines["NV CHIFFRE D'AFFAIRE"] = cost_of_delay_gb_airlines["CHIFFRE D'AFFAIRE COMPAGNIE"]\
                                                    - cost_of_delay_gb_airlines["TOTAL A PAYER"]

In [32]:
cost_of_delay_gb_airlines["%CHIFFRE D'AFFAIRE LOST"] = \
(cost_of_delay_gb_airlines["TOTAL A PAYER"]/cost_of_delay_gb_airlines["CHIFFRE D'AFFAIRE COMPAGNIE"])*100

In [33]:
cost_of_delay_gb_airlines

,COMPAGNIES,NOMBRE DE RETARD,CHIFFRE D'AFFAIRE COMPAGNIE,COUT DU RETARD,INDEMNITES A PAYER,NOMBRE DE CLIENTS PERDUS,COUT DES CLIENTS PERDUS,TOTAL A PAYER,NV CHIFFRE D'AFFAIRE,%CHIFFRE D'AFFAIRE LOST
0,Air Penguin,4,31064000000,26274,950550.0,118,106200,1083024.0,3.106292e+10,0.003486
1,Air Piloter Sans Les Mains,8,7651000000,22274,2556900.0,337,303300,2882474.0,7.648118e+09,0.037674
2,Always A Problem Flights,11,51000000000,52440,4311900.0,567,510300,4874640.0,5.099513e+10,0.009558
3,Bel Air,7,3671000000,23227,1055150.0,116,104400,1182777.0,3.669817e+09,0.032219
4,Better Take A Train Airlines,10,5056000000,33031,2173350.0,261,234900,2441281.0,5.053559e+09,0.048285
5,Corporate Overlord Airways,4,40579000000,36333,949800.0,125,112500,1098633.0,4.057790e+10,0.002707
6,Fliying Is Possible Inc.,1,2979000000,1898,393300.0,52,46800,441998.0,2.978558e+09,0.014837
7,IE 6.0 Flights,7,2660000000,19816,2133300.0,281,252900,2406016.0,2.657594e+09,0.090452
8,Morally Ambiguious Fligthts,5,6391000000,15985,1440600.0,190,171000,1627585.0,6.389372e+09,0.025467
9,Ne Va Pas Partout Airlines,10,6235000000,26911,3280800.0,434,390600,3698311.0,6.231302e+09,0.059315


In [38]:
fig = px.bar(cost_of_delay_gb_airlines,
             x="COMPAGNIES",
             y=["CHIFFRE D'AFFAIRE COMPAGNIE", "NV CHIFFRE D'AFFAIRE"],
             barmode='group',
             title="Repartition du Chiffre d'affaire et cout total du retard par Compagnie")
fig.show()

fig = px.bar(cost_of_delay_gb_airlines,
             x="COMPAGNIES",
             y=["CHIFFRE D'AFFAIRE COMPAGNIE", "TOTAL A PAYER", "INDEMNITES A PAYER","COUT DU RETARD", "COUT DES CLIENTS PERDUS"],
             barmode='group',
             title="Repartition du Chiffre d'affaire et cout total du retard par Compagnie")
fig.show()

In [36]:
import plotly.graph_objects as go

for idx, company in enumerate(cost_of_delay_gb_airlines["COMPAGNIES"]):
    labels = ["NV CHIFFRE D'AFFAIRE","TOTAL A PAYER"]
    values = [cost_of_delay_gb_airlines.iloc[idx]["NV CHIFFRE D'AFFAIRE"],
              cost_of_delay_gb_airlines.iloc[idx]["TOTAL A PAYER"]]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.2, 0])])
    #fig.update_traces(hole=.4, hoverinfo="label+percent+name")
    fig.update_layout(title_text=company)
    fig.show()

In [40]:
import plotly.graph_objects as go

for idx, company in enumerate(cost_of_delay_gb_airlines["COMPAGNIES"]):
    labels = ["NV CHIFFRE D'AFFAIRE","INDEMNITES A PAYER", "COUT DES CLIENTS PERDUS", "COUT DU RETARD"]
    values = [cost_of_delay_gb_airlines.iloc[idx]["NV CHIFFRE D'AFFAIRE"],
              cost_of_delay_gb_airlines.iloc[idx]["INDEMNITES A PAYER"],
             cost_of_delay_gb_airlines.iloc[idx]["COUT DES CLIENTS PERDUS"],
             cost_of_delay_gb_airlines.iloc[idx]["COUT DU RETARD"]]

    fig = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.2, 0])])
    #fig.update_traces(hole=.4, hoverinfo="label+percent+name")
    fig.update_layout(title_text=company)
    fig.show()

KeyError: 'key of type tuple not found and not a MultiIndex'

In [ ]:
#INDEMNITES A PAYER , COUT DES CLIENTS PERDUS, COUT DU RETARD